# CRAVAT treshold for training set
The purpose of this notebook is to calculate the treshold and accuracy of Cravat

#### Dependecies

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json
from sklearn.metrics import roc_curve, roc_auc_score

#### Load CSV file

In [22]:
df = pd.read_excel("training_set.vcf.CRAVAT_analysis.dev.loaceved.xls",skiprows=10,header=1)
df_actual=pd.read_excel("training_set_pathogenicity.xlsx")




In [23]:
frames = pd.concat([df['ID'],df['Chromosome'],df['Position'],df['Strand'],
                   df['Reference base(s)'],df['Alternate base(s)'],
                   df['ClinVar'],df['VEST p-value'].rename('VEST_p_value'),df['VEST FDR']], axis=1)

df_actual=pd.concat([df_actual['#Coordinate'].rename('ID'),df_actual['Pathogenicity'].rename('actual_Pathogenicity')],axis=1)

## Merge predicted and actual dataframe

In [24]:
frames=pd.merge(frames, df_actual, how='inner', on='ID')

#### Normilize Data (Likely_benign now Benign & Likely_pathogenic now Pathogenic)

In [25]:
frames.loc[(frames.actual_Pathogenicity=='Likely_benign'),'actual_Pathogenicity']= 'Benign'
frames.loc[(frames.actual_Pathogenicity=='Likely_pathogenic'),'actual_Pathogenicity']= 'Pathogenic'

#### Create temporary column

In [26]:
temp = list('B'*len(frames))
frames.insert(9, column='Predicted', value=temp)

### Initiate variables and empty lists

In [27]:
acum, tp, total, accuracy,pval=0,0,0,0,0
acum_acc,acum_pval,tn_lis,tp_lis,fp_lis,fn_lis=[],[],[],[],[],[]

## Calculate Accuracy and Treshold (pvalue)

In [28]:
for i in np.arange(0.05,1.,0.0001):
    i=round(i,4)
    frames.loc[(frames.VEST_p_value<=i), 'Predicted']= 'Pathogenic'
    frames.loc[(frames.VEST_p_value>i), 'Predicted']= 'Benign'
    frames

    tn=len(frames.loc[(frames.Predicted.astype(str)=='Benign')&(frames.actual_Pathogenicity.astype(str)=='Benign')])+0.0
    tp=len(frames.loc[(frames.Predicted.astype(str)=='Pathogenic')&(frames.actual_Pathogenicity.astype(str)=='Pathogenic')])+0.0

    fn=len(frames.loc[(frames.Predicted.astype(str)=='Benign')&(frames.actual_Pathogenicity.astype(str)=='Pathogenic')])+0.0
    fp=len(frames.loc[(frames.Predicted.astype(str)=='Pathogenic')&(frames.actual_Pathogenicity.astype(str)=='Benign')])+0.0
    t=tn+tp
    f=(fn+fp)
    total=t+f
    accuracy=((t/(total)*1.0)*1.0)
    
    tn_lis.append(tn)
    tp_lis.append(tp)
    fn_lis.append(fn)
    fp_lis.append(fp)
    
    acum_acc.append(accuracy)
    acum_pval.append(i)
    
    if(acum<accuracy):
        acum=accuracy
        pval=i
        

##### Round treshold 

In [29]:
pval=round(pval,2)
print(treshold "+str(pval))
acum=round(acum,2)
print ("accuracy "+str(acum))

SyntaxError: EOL while scanning string literal (<ipython-input-29-69a7ff009d6c>, line 2)

### Apply Best treshold

In [ ]:
frames.loc[(frames.VEST_p_value<=.09), 'Predicted']= 'Pathogenic'
frames.loc[(frames.VEST_p_value>.09), 'Predicted']= 'Benign'

##### Check How data is being orginized

In [ ]:
print("Benings in Actual: "+ str(len(frames.loc[(frames.actual_Pathogenicity=='Benign')])))
print("Pathogenic in Actual: "+str(len(frames.loc[(frames.actual_Pathogenicity=='Pathogenic')])))
print("Beningn in Predicted: "+str(len(frames.loc[(frames.Predicted=='Benign')])))
print("Pathogenic in Predicted: "+str(len(frames.loc[(frames.Predicted=='Pathogenic')])))

In [ ]:
sns.boxplot(x='actual_Pathogenicity',y='VEST_p_value',data=frames,order=["Pathogenic", "Benign"],)


In [ ]:
sns.boxplot(x='Predicted',y='VEST_p_value',data=frames,order=["Pathogenic", "Benign"])

## Accuracy & P-value

In [ ]:
print("Accuracy: "+str(acum))
print("Pvalue: " +str(pval))

### Plot: true_pos, true_neg, false_neg, false_pos

In [ ]:
plt.figure(figsize=(16, 8))
plt.plot(acum_pval, tn_lis, c='b', label='True Negative')
plt.plot(acum_pval, tp_lis, c='g', label='True Postive')
plt.plot(acum_pval, fn_lis, c='r', label='False Negative')
plt.plot(acum_pval, fp_lis, c='y', label='False Positive')
plt.legend()
plt.xlabel("Threshold")
plt.ylabel("Data Amount")
plt.title("Threshold vs DataAmount by ...");
# plt.axhline(max_y, c='r')
# plt.axvline(max_x, c='g')
plt.savefig("thresVsData_training.pdf")
plt.savefig("thresVsData_training.png")

## Plot: accuracy vs pvalue 

In [ ]:
max_idx = np.argmax(acum_acc)
max_x = acum_pval[max_idx]
max_y = acum_acc[max_idx]

In [ ]:
plt.figure(figsize=(16, 8))
plt.plot(acum_pval, acum_acc)
plt.axhline(max_y, c='r')
plt.axvline(max_x, c='g')
plt.savefig("thresData_training.pdf")
plt.savefig("thresData_training.png")

In [ ]:
# acum_acc

In [ ]:
# np.linspace(0.0, 1.0, 1000 )

In [ ]:
# acum=round(acum,2)
# acum

## Box plot

In [ ]:
sns.boxplot(x='actual_Pathogenicity',y='VEST_p_value',data=frames,order=["Pathogenic", "Benign"])

In [ ]:
sns.boxplot(x='Predicted',y='VEST_p_value',data=frames,order=["Pathogenic", "Benign"])

In [ ]:
rev_frames=frames

rev_frames['VEST_p_value']=1-rev_frames['VEST_p_value']
rev_frames.loc[(rev_frames.VEST_p_value>=pval), 'Predicted']= 'Pathogenic'
rev_frames.loc[(rev_frames.VEST_p_value<pval), 'Predicted']= 'Benign'
print(len(rev_frames.loc[(rev_frames.Predicted=='Benign')]))
print(len(rev_frames.loc[(rev_frames.Predicted=='Pathogenic')]))

In [ ]:
sns.boxplot(x='actual_Pathogenicity',y='VEST_p_value',data=rev_frames,order=["Pathogenic", "Benign"])

In [ ]:
sns.boxplot(x='Predicted',y='VEST_p_value',data=rev_frames,order=["Pathogenic", "Benign"])

## Roc plot

In [ ]:
fpr, tpr, thr = roc_curve(frames.actual_Pathogenicity,frames.VEST_p_value, pos_label='Pathogenic')

In [ ]:
plt.plot(fpr, tpr)
#plt.plot(thr, thr,'.-' 

## Area under Curve

In [ ]:
y_true = np.ones(len(frames.actual_Pathogenicity))
y_true[frames.actual_Pathogenicity == 'Benign'] = 0
roc_auc_score(y_true,frames.VEST_p_value)

In [ ]:
sns.violinplot(x='actual_Pathogenicity',y='VEST_p_value',data=rev_frames,order=["Pathogenic", "Benign"],cut=0)

In [ ]:
sns.violinplot(x='Predicted',y='VEST_p_value',data=rev_frames,order=["Pathogenic", "Benign"],cut=0)